Example usage of the provided "cython_peg.py" parsing file.  This code is not mature/complex enough to be a python module.  
The parsing will probably need to be adjusted for specific project needs.  The pyparsing syntax is dense but powerful.  
It will take some document reading to learn the pyparsing syntax and make edits.

In [ ]:
import cythonpeg as cp
from pathlib import Path
from textwrap import indent

# imports are tansfered now
# import template for introduced cython type conversions
import_template = "\n".join(
    [
        "# additional static imports",
        "import numpy as np",
        "float64 = float32 = double = long = longlong = float",
        "uint64 = uint32 = uint16 = uint8 = short = int",
        "\n",
    ]
)

cp.set_indent("    ")


def custom_cython_2_python_type_partial(type_str: str) -> str:
    # cython pointer syntax '*' included as type_str
    if type_str.replace("*", "") in ["float32", "float64", "double"]:
        return "float"

    elif type_str.replace("*", "") in ["uin32", "uint64", "int32", "int64"]:
        return "int"

    elif type_str.replace("*", "") in ["bint"]:
        return "bool"

    else:
        return type_str


def custom_cython_2_python_type_complete(type_str: str) -> str:
    if ":" in type_str:
        return "np.ndarray"
    return type_str


cp.set_type_converter_partial(custom_cython_2_python_type_partial)
cp.set_type_converter_complete(custom_cython_2_python_type_complete)

files = Path(r"../tests/cython").glob("*.pyx")
for file in files:
    print(f"parsing file: {file}")

    # read stub file
    with open(file, mode="r") as f:
        input_string = f.read()

    stub_file, unparsed_lines = cp.cython_string_2_stub(input_string)

    print(f"Percentage Parsed: {1 - len(unparsed_lines) / len(input_string)}")
    print(f"    Unparsed Lines:")
    print(indent(unparsed_lines, " " * 8))

    # save the type file
    with open(file.with_suffix(".pyi"), mode="w") as f:
        f.write(import_template + stub_file)

    # print output
    print(f"    Stub Output:")
    print(indent(import_template + stub_file, " " * 8))


parsing file: ../tests/cython/enum_a.pyx
Percentage Parsed: 1.0
    Unparsed Lines:

    Stub Output:
        # additional static imports
        import numpy as np
        float64 = float32 = double = long = longlong = float
        uint64 = uint32 = uint16 = uint8 = short = int

        from enum import Enum

        class State(Enum):
               state1=1
               state2=2

parsing file: ../tests/cython/class_c.pyx
Percentage Parsed: 1.0
    Unparsed Lines:

    Stub Output:
        # additional static imports
        import numpy as np
        float64 = float32 = double = long = longlong = float
        uint64 = uint32 = uint16 = uint8 = short = int

        from typing import List

        class OuterClass(object):
               """documentation of outer class"""

               def method2(self, a: float, b: float) -> List:
                      """outer method"""
                      ...

        class InnerClass(object):
               """document of inner class"""

